# Interactive Surrogate Modelling

In this notebook we:
+ define a problem consisting of parameters to vary with ranges and an objective function
+ sample the problem space
+ define an Evaluator by linking the problem to an EnergyPlus model
+ run the Evaluator for the samples
+ train a surrogate model over the samples
+ explore the design space using an interactive plot that queries the surrogate model

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from besos import eppy_funcs as ef, sampling
from besos.evaluator import EvaluatorEP
from besos.parameters import FieldSelector, Parameter, RangeParameter
from besos.problem import EPProblem
from ipywidgets import FloatSlider, interact
from matplotlib import pyplot as plt
from sklearn import linear_model, pipeline
from sklearn.preprocessing import StandardScaler

## Parameters and Objectives
We start by defining `Parameters` that govern how we would like to modify the building.
Here they are the solar heat gain coefficient and the lighting power density.

In [ ]:
parameters = [
    Parameter(
        FieldSelector(
            object_name="NonRes Fixed Assembly Window",
            field_name="Solar Heat Gain Coefficient",
        ),
        value_descriptor=RangeParameter(0.01, 0.99),
    ),
    Parameter(
        FieldSelector("Lights", "*", "Watts per Zone Floor Area"),
        value_descriptor=RangeParameter(8, 12),
        name="Lights Watts/Area",
    ),
]

Now, we specify that we would like to optimize for electricity use of the entire facility. We then bundle all of this information together.

In [ ]:
objectives = ["Electricity:Facility"]

problem = EPProblem(parameters, objectives)

## Sampling

We need to generate samples on which to train a surrogate model.
The `sampler` uses Latin Hypercube Sampling to distribute the samples in the design space defined by the `Parameters`.
We can check how well distributed they are using `describe()`.

In [ ]:
inputs = sampling.dist_sampler(sampling.lhs, problem, 5)
inputs.describe()  # properties of the sample set

## Building model
Next we load an EnergyPlus model. The example is a small office, or we could pass in the IDF filename.

In [ ]:
building = ef.get_building()

## Evaluator
We now define and run the Evaluator, which will run the EnergyPlus instances with the different inputs generated in the previous cell.

In [ ]:
evaluator = EvaluatorEP(problem, building)
train = evaluator.df_apply(inputs, keep_input=True)
train.head()  # first 5 lines

We can save results at this point:

In [ ]:
train.to_pickle("samples.p")

And load them again later:

In [ ]:
train = pd.read_pickle("samples.p")

## Exploring the training data
Let's look at the correlation amongst the inputs and the output using a built-in [Seaborn](http://seaborn.pydata.org/) plot:

In [ ]:
g = sns.pairplot(train)

Here's a plot of the _design space_ defined by the samples.

In [ ]:
x, y, obj = problem.names()
train.plot.hexbin(x=x, y=y, C=obj, cmap="plasma", sharex=False, gridsize=50)

## Fitting a surrogate model

Now we can train the model using a ScikitLearn pipeline:

In [ ]:
model = pipeline.make_pipeline(StandardScaler(), linear_model.Ridge())
model.fit(train[[x, y]].values, train[obj].values)

## Running the surrogate model

We can query the model at a specific point in the design space:

In [ ]:
model.predict([[0.5, 10]])

Or for a `dataframe` made from a `dictionary` of points:

In [ ]:
points = pd.DataFrame(data={"SHGC": [0, 1], "LPD": [8, 12]})
model.predict(points)

Next we define a function for querying the surrogate model across the domain at a given density, and plot the result.

In [ ]:
def run_model(model, density):
    p1 = problem.inputs[0].value_descriptor
    a = np.linspace(p1.min, p1.max, density)
    p2 = problem.inputs[1].value_descriptor
    b = np.linspace(p2.min, p2.max, density)
    plot_data = pd.DataFrame(
        np.transpose([np.tile(a, len(b)), np.repeat(b, len(a))]),
        columns=problem.names("inputs"),
    )
    return pd.concat([plot_data, pd.Series(model.predict(plot_data))], axis=1)


density = 200
df = run_model(model, density)
df.plot.hexbin(x=x, y=y, C=0, cmap="plasma", gridsize=density, sharex=False)

## Interactive surrogate
Finally we make an interactive plot using `ipywidgets`, which executes the model every time a slider is moved.

In [ ]:
# get the min and max of the two variables
(min1, max1), (min2, max2) = (
    (float(p.value_descriptor.min), p.value_descriptor.max) for p in problem.inputs
)
# get the data to plot in the background
df = run_model(model, 100)

# define a wrapper to be queried by interact
def model_wrapper(v1, v2):
    df.plot.hexbin(
        x=x, y=y, C=0, cmap="plasma", gridsize=30, sharex=False
    )  # background plot
    x_lims = plt.xlim()
    y_lims = plt.ylim()
    plt.scatter(x=v1, y=v2)  # plot the marker
    plt.xlim((x_lims))
    plt.ylim((y_lims))

    value = (
        model.predict([[v1, v2]])[0] / 3.6e6
    )  # find the current value and convert to kWh
    output = (
        "Electricity use = " + str(value.round()) + " kWh"
    )  # return the string to display
    return output


# make the interactive plot with two sliders
continuous_update = False
interact(
    model_wrapper,
    v1=FloatSlider(
        min=min1,
        max=max1,
        value=(min1 + max1) / 2,
        description="SHGC",
        step=0.01,
        continuous_update=continuous_update,
    ),
    v2=FloatSlider(
        min=min2,
        max=max2,
        value=(min2 + max2) / 2,
        description="LPD [W/m2]",
        continuous_update=continuous_update,
    ),
)